# 1. 自动识别

In [ ]:
import os

In [ ]:
list_file = list(os.walk('./input'))[0][2]
if '.DS_Store' in list_file:
    list_file.remove('.DS_Store')
if '0.null.txt' in list_file:
    list_file.remove('0.null.txt')
    
for file in list_file:
    if '整理好的替代料总表' in file:
        alternate = './input/' + file
    elif '改价表' in file:
        price = './input/' + file
    elif '替代料总表' in file and '整理好的' not in file:
        combination = './input/' + file
    elif 'MRP' in file:
        mrp = './input/' + file
    elif '划分' in file:
        group = './input/' + file
    elif '断货改价' in file:
        last = './input/' + file

print('alternate:     ' + alternate)
print('price:         ' + price)
print('combination:   ' + combination)
print('mrp:           ' + mrp)
print('group:         ' + group)
print('last:          ' + last)

# 2. 在库在途计算
## 2.1. 替代料计算

In [ ]:
import pandas as pd

In [ ]:
df_alternate = pd.read_excel(alternate,
                             header=0,
                             dtype=str).fillna('')

df_alternate['row'] = [i+1 for i in range(len(df_alternate))]

df_alternate

In [ ]:
# 替代料表变形
list_df = []
for column in df_alternate.columns.tolist()[:-1]:
    df_temp =  df_alternate[['主料SKU', column, 'row']]
    df_temp.columns = ['主料', 'SKU', 'row']
    df_temp = df_temp[df_temp['SKU'] != '']
    list_df.append(df_temp)

df_alternate = pd.concat(list_df, ignore_index=True)

df_alternate.to_excel('./output/1.position.xlsx', index=False)

df_alternate

In [ ]:
df_price = pd.read_excel(price,
                         sheet_name=0,
                         header=0,
                         names=['SKU', '在途', '在库'],
                         usecols='A, E, F',
                         dtype=str).fillna({'SKU': '',
                                            '在途': 0,
                                            '在库': 0})

df_price = df_price[df_price['SKU'] != ''].reset_index(drop=True)

df_price['在库'] = df_price['在库'].astype(int)
df_price['在途'] = df_price['在途'].astype(int)

# 处理后缀
# -N：原单非质量、无理由 等问题的售后退件
# -NC：原单投诉质量问题未核实、货值比较大的订单；原单非质量问题但产品略有小瑕疵（一丢丢脏）
# -D：原单不是质量问题，但退件有缺失一些小部件
df_price['SKU'] = [sku.strip()[:-3] if sku.strip().endswith('-NC') else sku.strip()[:-2] if sku.strip().endswith('-N') else sku.strip()[:-2] if sku.strip().endswith('-D') else sku.strip() for sku in df_price['SKU'].tolist()]

df_price = df_price.groupby([df_price['SKU']], as_index=False).sum()

df_price

In [ ]:
print(df_alternate.columns)
print(df_price.columns)

# 找出有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna(0)


df_temp

In [ ]:
# 合并有替代料关系的数据
df_temp = df_temp[['row', '在库', '在途']]
df_temp = df_temp.groupby([df_temp['row']], as_index=False).sum()

df_temp

In [ ]:
print(df_alternate.columns)
print(df_temp.columns)

# 更新有替代料关系的数据
df_temp = pd.merge(left=df_alternate,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp

In [ ]:
print(df_price.columns)
print(df_alternate.columns)

# 找出没有替代料关系的数据
df_temp_2 = pd.merge(left=df_price,
                     right=df_alternate,
                     how='left',
                     left_on='SKU',
                     right_on='SKU').fillna('')

df_temp_2 = df_temp_2[df_temp_2['row'] == '']
df_temp_2['主料'] = df_temp_2['SKU'].tolist()
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途']]

df_temp_2

In [ ]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)
df_price['new_row'] = [i+1 for i in range(len(df_price), len(df_price) * 2)]
df_price['row'] = [row if row != '' else new_row for row, new_row in zip(df_price['row'].tolist(), df_price['new_row'].tolist())]
df_price = df_price[['主料', 'SKU', 'row', '在库', '在途']]

df_price

## 2.2. 组合计算

In [ ]:
df_combination = pd.read_excel(combination,
                               header=1,
                               dtype=str)[['主料SKU', '数量', 'SKU1', 'SKU2', 'SKU3', 'SKU4', 'SKU5']].fillna('')

df_combination['数量'] = [amount.strip() for amount in df_combination['数量'].tolist()]
df_combination = df_combination[(df_combination['数量'] != '') & (df_combination['数量'] != '1')].reset_index(drop=True)

df_combination

In [ ]:
print(df_combination.columns)
print(df_price.columns)

df_combination = pd.merge(left=df_combination,
                          right=df_price,
                          how='left',
                          left_on='主料SKU',
                          right_on='SKU').fillna('')

df_combination

In [ ]:
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU1',
                   right_on='SKU',
                   suffixes=('', '_1')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU2',
                   right_on='SKU',
                   suffixes=('', '_2')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU3',
                   right_on='SKU',
                   suffixes=('', '_3')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU4',
                   right_on='SKU',
                   suffixes=('', '_4')).fillna('')
df_combination = pd.merge(left=df_combination,
                   right=df_price[['SKU', '在库', '在途']],
                   how='left',
                   left_on='SKU5',
                   right_on='SKU',
                   suffixes=('', '_5')).fillna('')

df_combination

In [ ]:
from tqdm import tqdm
import math

In [ ]:
for i in tqdm(range(len(df_combination)), desc='Progress', ncols=77):
    list_amount = [int(amount.strip()) for amount in df_combination.loc[i, '数量'].split('+')]
    list_stock = []
    list_way = []
    for j in range(len(list_amount)):
        list_stock.append(0 if df_combination.loc[i, '在库_'+str(j+1)] == '' else df_combination.loc[i, '在库_'+str(j+1)] / list_amount[j])
        list_way.append(0 if df_combination.loc[i, '在途_'+str(j+1)] == '' else df_combination.loc[i, '在途_'+str(j+1)] / list_amount[j])
    df_combination.loc[i, '组合在库'] = math.floor(min(list_stock))
    df_combination.loc[i, '组合在途'] = math.floor(min(list_way))
        
df_combination = df_combination[['主料SKU', 'row', '组合在库', '组合在途']]
df_combination = df_combination.sort_values(by=['组合在库', '组合在途'], ascending=[False, False]).reset_index(drop=True)

df_combination

In [ ]:
# 找出有替代料关系的数据
df_temp = df_combination[df_combination['row'] != ''].reset_index(drop=True)
df_temp = df_temp.drop_duplicates(['row'], keep='first', ignore_index=True)

print(df_price.columns)
print(df_combination.columns)

df_temp = pd.merge(left=df_price,
                   right=df_temp,
                   how='left',
                   left_on='row',
                   right_on='row').fillna(0)

df_temp = df_temp[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

df_temp

In [ ]:
from datetime import datetime

In [ ]:
# 找出没有替代料关系的数据
df_temp_2 = df_combination[df_combination['row'] == ''].reset_index(drop=True)
df_temp_2 = df_temp_2.drop_duplicates(['主料SKU'], keep='first', ignore_index=True)
df_temp_2['row'] = [i+1 for i in range(len(df_price) * 2, len(df_price) * 2 + len(df_temp_2))]
df_temp_2['主料'] = df_temp_2['主料SKU'].tolist()
df_temp_2['SKU'] = df_temp_2['主料SKU'].tolist()
df_temp_2['在库'] = 0
df_temp_2['在途'] = 0
df_temp_2 = df_temp_2[['主料', 'SKU', 'row', '在库', '在途', '组合在库', '组合在途']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU', '组合在库', '组合在途']].to_excel('./output/warning-1.组合缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

In [ ]:
# 整合数据
df_price = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_price.to_excel('./output/2.stock.xlsx', index=False)

df_price

# 3. MRP整理

In [ ]:
df_mrp = pd.read_excel(mrp,
                       header=1,
                       names=['SKU', '品类', '层级', '身份', '未售出周数', '在途总数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1', 'MRP周销'],
                       usecols='B, F, G, I, K, P, V:AB, AD',
                       dtype=str).fillna('')

df_group = pd.read_excel(group,
                         header=0,
                         dtype=str).fillna('')

df_mrp = pd.merge(left=df_mrp,
                  right=df_group,
                  how='left',
                  left_on='品类',
                  right_on='品类').fillna('')

df_mrp = df_mrp[df_mrp['品类负责人'] == 'Ada'].reset_index(drop=True)

df_mrp

In [ ]:
print(df_mrp.columns)
print(df_price.columns)

df_mrp = pd.merge(left=df_mrp,
                  right=df_price,
                  how='left',
                  left_on='SKU',
                  right_on='SKU').fillna('')

df_mrp['row'] = ['' if row == '' else int(row) for row in df_mrp['row'].tolist()]

df_mrp.to_excel('./output/3.mrp.xlsx', index=False)

df_mrp

In [ ]:
df_temp = df_mrp.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

In [ ]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-1.mrp重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

In [ ]:
import sys

In [ ]:
if len(df_temp_1) != 0:
    sys.exit()

In [ ]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-2.mrp缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

# 4. last与MRP整合

In [ ]:
df_last = pd.read_excel(last,
                        header=1,
                        usecols='A, C:L, AE:AO, AQ:AU, AX:AY',
                        dtype=str).fillna('')

df_last['上次序号'] = [int(i.strip()) for i in df_last['序号'].tolist()]

print(df_last.columns)

df_last

In [ ]:
print(df_last.columns)
print(df_price.columns)

df_last = pd.merge(left=df_last,
                   right=df_price,
                   how='left',
                   left_on='SKU',
                   right_on='SKU').fillna('')

df_last['row'] = ['' if row == '' else int(row) for row in df_last['row'].tolist()]

df_last

In [ ]:
df_temp = df_last.copy()
df_temp['times_row'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]
df_temp['times_sku'] = [df_temp['SKU'].tolist().count(sku) for sku in df_temp['SKU'].tolist()]

df_temp

In [ ]:
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times_row'] != 1)][['SKU', 'row', 'times_row']]
df_temp_2 = df_temp[(df_temp['row'] == '') & (df_temp['times_sku'] != 1)][['SKU', 'row', 'times_sku']]
df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

if len(df_temp_1) != 0:
    df_temp_1 = df_temp_1.sort_values(by=['row', 'SKU'], ascending=[True, True]).reset_index(drop=True)
    df_temp_1.to_excel('./output/error-2.last重复出现-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_1

In [ ]:
if len(df_temp_1) != 0:
    sys.exit()

In [ ]:
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU']]

if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2.to_excel('./output/warning-3.last缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

In [ ]:
print(df_last.columns)
print(df_mrp.columns)

# 找出有库存信息的数据
df_temp = pd.merge(left=df_last[df_last['row'] != ''],
                   right=df_mrp,
                   how='left',
                   left_on='row',
                   right_on='row',
                   suffixes=('', '_mrp')).fillna('')

df_temp['SKU'] = [sku if sku_mrp == '' else sku_mrp for sku, sku_mrp in zip(df_temp['SKU'].tolist(), df_temp['SKU_mrp'].tolist())]
df_temp['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp['品类负责人_mrp'].tolist()]
list_column = df_temp.columns.tolist()
list_column.remove('SKU_mrp')
list_column.remove('品类负责人_mrp')
list_column.remove('主料_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp = df_temp[list_column]

df_temp

In [ ]:
print(df_last.columns)
print(df_mrp.columns)

# 找出没有库存信息的数据
df_temp_2 = pd.merge(left=df_last[df_last['row'] == ''],
                     right=df_mrp,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_mrp')).fillna('')

df_temp_2['在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['在库'].tolist(), df_temp_2['在库_mrp'].tolist())]
df_temp_2['在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['在途'].tolist(), df_temp_2['在途_mrp'].tolist())]
df_temp_2['组合在库'] = [stock if stock_mrp == '' else stock_mrp for stock, stock_mrp in zip(df_temp_2['组合在库'].tolist(), df_temp_2['组合在库_mrp'].tolist())]
df_temp_2['组合在途'] = [way if way_mrp == '' else way_mrp for way, way_mrp in zip(df_temp_2['组合在途'].tolist(), df_temp_2['组合在途_mrp'].tolist())]
df_temp_2['Note'] = ['' if charge == 'Ada' else 'last only, not in MRP' for charge in df_temp_2['品类负责人_mrp'].tolist()]
list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_mrp')
list_column.remove('主料_mrp')
list_column.remove('row_mrp')
list_column.remove('在库_mrp')
list_column.remove('在途_mrp')
list_column.remove('组合在库_mrp')
list_column.remove('组合在途_mrp')
df_temp_2 = df_temp_2[list_column]

df_temp_2

In [ ]:
# 整合数据
df_temp = pd.concat([df_temp, df_temp_2], ignore_index=True)

df_temp

In [ ]:
print(df_mrp.columns)
print(df_last.columns)

df_temp_2 = pd.merge(left=df_mrp,
                     right=df_last,
                     how='left',
                     left_on='SKU',
                     right_on='SKU',
                     suffixes=('', '_last')).fillna('')

df_temp_2 = df_temp_2[df_temp_2['序号'] == ''].reset_index(drop=True)
for i in range(len(df_temp_2)):
    if df_temp_2.loc[i, 'SKU'] not in df_temp['SKU'].tolist():
        df_temp_2.loc[i, 'Note'] = 'MRP only, not in last'

if len(df_temp_2) > 0:
    df_temp_2 = df_temp_2[df_temp_2['Note'] == 'MRP only, not in last'].reset_index(drop=True)

df_temp_2['品类负责人'] = df_temp_2['品类负责人_last'].tolist()

list_column = df_temp_2.columns.tolist()
list_column.remove('品类负责人_last')
list_column.remove('主料_last')
list_column.remove('row_last')
list_column.remove('在库_last')
list_column.remove('在途_last')
list_column.remove('组合在库_last')
list_column.remove('组合在途_last')
df_temp_2 = df_temp_2[list_column]

df_temp_2

In [ ]:
df = pd.concat([df_temp, df_temp_2], ignore_index=True)

df

# 5. 其他计算

In [ ]:
df_missing = pd.read_excel(price,
                           sheet_name=1,
                           header=0,
                           names=['SKU', '数量'],
                           usecols='D, F',
                           dtype=str).fillna('')

df_missing = df_missing[df_missing['SKU'] != ''].reset_index(drop=True)

df_missing['数量'] = df_missing['数量'].astype(int)
df_missing = df_missing.groupby([df_missing['SKU']], as_index=False).sum()

df_missing

In [ ]:
print(df_missing.columns)
print(df_price.columns)

df_missing = pd.merge(left=df_missing,
                      right=df_price,
                      how='left',
                      left_on='SKU',
                      right_on='SKU').fillna('')

df_missing

In [ ]:
df_temp = df_missing.copy()
df_temp['times'] = [df_temp['row'].tolist().count(row) for row in df_temp['row'].tolist()]

df_temp

In [ ]:
# 找出有库存信息的数据
df_temp_1 = df_temp[(df_temp['row'] != '') & (df_temp['times'] != 1)][['row', '数量']]
df_temp_1 = df_temp_1.groupby([df_temp_1['row']], as_index=False).sum()
df_temp_2 = df_temp[(df_temp['row'] != '') & (df_temp['times']) == 1][['row', '数量']]

df_temp_1 = pd.concat([df_temp_1, df_temp_2], ignore_index=True)

df_temp_1

In [ ]:
print(df.columns)
print(df_temp_1.columns)

df = pd.merge(left=df,
              right=df_temp_1,
              how='left',
              left_on='row',
              right_on='row').fillna(0)

df

In [ ]:
# 找出没有库存信息的数据
df_temp_2 = df_temp[df_temp['row'] == ''][['SKU', '数量']]
                                          
if len(df_temp_2) != 0:
    df_temp_2 = df_temp_2.sort_values(by=['SKU'], ascending=[True]).reset_index(drop=True)
    df_temp_2[['SKU']].to_excel('./output/warning-4.无record缺失信息-'+ datetime.now().strftime("%Y%m%d_%H%M%S") + '.xlsx', index=False)

df_temp_2

In [ ]:
print(df.columns)
print(df_temp_2.columns)

df = pd.merge(left=df,
              right=df_temp_2,
              how='left',
              left_on='SKU',
              right_on='SKU',
              suffixes=('_1', '_2')).fillna(0)

df

In [ ]:
df['无Record ID'] = [amount_1 + amount_2 for amount_1, amount_2 in zip(df['数量_1'].tolist(), df['数量_2'].tolist())]
df['在库总数'] = [stock_1 + stock_2 for stock_1, stock_2 in zip(df['在库'].tolist(), df['组合在库'].tolist())]
df['在途总数'] = [way_1 + way_2 for way_1, way_2 in zip(df['在途'].tolist(), df['组合在途'].tolist())]

list_column = df.columns.tolist()
list_column.remove('主料')
list_column.remove('row')
list_column.remove('在库')
list_column.remove('组合在库')
list_column.remove('在途')
list_column.remove('组合在途')
list_column.remove('数量_1')
list_column.remove('数量_2')
df = df[list_column]

df

In [ ]:
df['累计涨价'] = ['=SUM(F' + str(i+3) + ':L' + str(i+3) + ')' for i in range(len(df))]
df['近7周平均销量'] = ['=IFERROR(ROUNDUP(AVERAGE(S' + str(i+3) + ':Y' + str(i+3) + '),1),"-")' for i in range(len(df))]
df['改价'] = ['=IFERROR(IF(AC' + str(i+3) + '-AE' + str(i+3) + '>0,"海运上架",IF(AC' + str(i+3) + '<=0,"断货","")),"")' for i in range(len(df))]
df['实时库存'] = ['=IFERROR(AD' + str(i+3) + '-AB' + str(i+3) + ',"")' for i in range(len(df))]
df['Week-Sale'] = ['=SUM(AI' + str(i+3) + ':AO' + str(i+3) + ')' for i in range(len(df))]
df['可周转数'] = ['=IFERROR(AC' + str(i+3) + '/AP' + str(i+3) + ',"")' for i in range(len(df))]
df['干预判断'] = ['=IFERROR(IFS(AV' + str(i+3) + '<4,"需要干预",AND(AV' + str(i+3) + '>4,AX' + str(i+3) + '<>""),"恢复干预"),"")' for i in range(len(df))]
df['改价记录1'] = ['=AR' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]
df['本周改价记录1'] = ['=AS' + str(i+3) + '&AY' + str(i+3) for i in range(len(df))]

df

In [ ]:
list_column = df.columns.tolist()
df = df[['序号', '层级', '在线SKU', 'SKU', '每周一大账号售价', '第一涨价', '第二涨价', '第三涨价', '第四涨价', '第五涨价', '第六涨价', '第七涨价', '累计涨价',
         '品类', '身份', 'MRP周销', '近7周平均销量', '未售出周数', 'W7', 'W6', 'W5', 'W4', 'W3', 'W2', 'W1',
         '在途总数', '改价', '无Record ID', '实时库存', '在库总数'] +
        list_column[11:22] +
        ['Week-Sale', '周销干预', '改价记录', '本周改价记录', '最新断货时间', '品类负责人', '可周转数', '干预判断', '干预记录',
         '提价', '改价记录1', '本周改价记录1', 'Note', '上次序号']]
list_column = df.columns.tolist()
list_column[list_column.index('在库总数')] = '当日库存'
df.columns = list_column

df.sort_values(by=['上次序号', 'SKU'], ascending=[True, True]).reset_index(drop=True)
df['序号'] = [i+1 for i in range(len(df))]

df

In [ ]:
for i in tqdm(range(len(df)), desc='Progress', ncols=77):
    try:
        df.loc[i, '每周一大账号售价'] = float(df.loc[i, '每周一大账号售价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第一涨价'] = float(df.loc[i, '第一涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第二涨价'] = float(df.loc[i, '第二涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第三涨价'] = float(df.loc[i, '第三涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第四涨价'] = float(df.loc[i, '第四涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第五涨价'] = float(df.loc[i, '第五涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第六涨价'] = float(df.loc[i, '第六涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, '第七涨价'] = float(df.loc[i, '第七涨价'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'MRP周销'] = float(df.loc[i, 'MRP周销'].strip())
    except:
        pass
    
    try:
        df.loc[i, '未售出周数'] = int(df.loc[i, '未售出周数'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W7'] = int(df.loc[i, 'W7'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W6'] = int(df.loc[i, 'W6'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W5'] = int(df.loc[i, 'W5'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W4'] = int(df.loc[i, 'W4'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W3'] = int(df.loc[i, 'W3'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W2'] = int(df.loc[i, 'W2'].strip())
    except:
        pass
    
    try:
        df.loc[i, 'W1'] = int(df.loc[i, 'W1'].strip())
    except:
        pass
    
    try:
        df.loc[i, '在途总数'] = int(df.loc[i, '在途总数'].strip())
    except:
        pass
    
    try:
        df.loc[i, '无Record ID'] = int(df.loc[i, '无Record ID'].strip())
    except:
        pass
    
    try:
        df.loc[i, '当日库存'] = int(df.loc[i, '当日库存'].strip())
    except:
        pass
    
    try:
        df.iloc[i, 30] = int(df.iloc[i, 30].strip())
    except:
        pass
    
    try:
        df.iloc[i, 31] = int(df.iloc[i, 31].strip())
    except:
        pass
    
    try:
        df.iloc[i, 32] = int(df.iloc[i, 32].strip())
    except:
        pass
    
    try:
        df.iloc[i, 33] = int(df.iloc[i, 33].strip())
    except:
        pass
    
    try:
        df.iloc[i, 34] = int(df.iloc[i, 34].strip())
    except:
        pass
    
    try:
        df.iloc[i, 35] = int(df.iloc[i, 35].strip())
    except:
        pass
    
    try:
        df.iloc[i, 36] = int(df.iloc[i, 36].strip())
    except:
        pass
    
    try:
        df.iloc[i, 37] = int(df.iloc[i, 37].strip())
    except:
        pass
    
    try:
        df.iloc[i, 38] = int(df.iloc[i, 38].strip())
    except:
        pass
    
    try:
        df.iloc[i, 39] = int(df.iloc[i, 39].strip())
    except:
        pass
    
    try:
        df.iloc[i, 40] = int(df.iloc[i, 40].strip())
    except:
        pass
    
df.to_excel('./output/pm-colorless.xlsx', index=False, startrow=1)

df

# 6. 格式更改

In [ ]:
from openpyxl import load_workbook

In [ ]:
wb_last = load_workbook(last)
ws_last = wb_last.worksheets[0]

wb_this = load_workbook('./output/pm-colorless.xlsx')
ws_this = wb_this.worksheets[0]

In [ ]:
coef = 1.15
ws_this.column_dimensions['A'].width = coef * 6
ws_this.column_dimensions['B'].width = coef * 9
ws_this.column_dimensions['C'].width = coef * 15
ws_this.column_dimensions['D'].width = coef * 15
ws_this.column_dimensions['E'].width = coef * 9
ws_this.column_dimensions['F'].width = coef * 8
ws_this.column_dimensions['G'].width = coef * 8
ws_this.column_dimensions['H'].width = coef * 8
ws_this.column_dimensions['I'].width = coef * 8
ws_this.column_dimensions['J'].width = coef * 8
ws_this.column_dimensions['K'].width = coef * 8
ws_this.column_dimensions['L'].width = coef * 8
ws_this.column_dimensions['M'].width = coef * 9
ws_this.column_dimensions['N'].width = coef * 9
ws_this.column_dimensions['O'].width = coef * 9
ws_this.column_dimensions['P'].width = coef * 7
ws_this.column_dimensions['Q'].width = coef * 7
ws_this.column_dimensions['R'].width = coef * 5
ws_this.column_dimensions['S'].width = coef * 5
ws_this.column_dimensions['T'].width = coef * 5
ws_this.column_dimensions['U'].width = coef * 5
ws_this.column_dimensions['V'].width = coef * 5
ws_this.column_dimensions['W'].width = coef * 5
ws_this.column_dimensions['X'].width = coef * 5
ws_this.column_dimensions['Y'].width = coef * 5
ws_this.column_dimensions['Z'].width = coef * 7
ws_this.column_dimensions['AA'].width = coef * 7
ws_this.column_dimensions['AB'].width = coef * 10
ws_this.column_dimensions['AC'].width = coef * 10
ws_this.column_dimensions['AD'].width = coef * 10
ws_this.column_dimensions['AE'].width = coef * 10
ws_this.column_dimensions['AF'].width = coef * 10
ws_this.column_dimensions['AG'].width = coef * 10
ws_this.column_dimensions['AH'].width = coef * 10
ws_this.column_dimensions['AI'].width = coef * 10
ws_this.column_dimensions['AJ'].width = coef * 10
ws_this.column_dimensions['AK'].width = coef * 10
ws_this.column_dimensions['AL'].width = coef * 10
ws_this.column_dimensions['AM'].width = coef * 10
ws_this.column_dimensions['AN'].width = coef * 10
ws_this.column_dimensions['AO'].width = coef * 10
ws_this.column_dimensions['AP'].width = coef * 8
ws_this.column_dimensions['AQ'].width = coef * 10
ws_this.column_dimensions['AR'].width = coef * 10
ws_this.column_dimensions['AS'].width = coef * 10
ws_this.column_dimensions['AT'].width = coef * 10
ws_this.column_dimensions['AU'].width = coef * 8
ws_this.column_dimensions['AV'].width = coef * 8
ws_this.column_dimensions['AW'].width = coef * 8
ws_this.column_dimensions['AX'].width = coef * 8
ws_this.column_dimensions['AY'].width = coef * 8
ws_this.column_dimensions['AZ'].width = coef * 8
ws_this.column_dimensions['BA'].width = coef * 8
ws_this.column_dimensions['BB'].width = coef * 10
ws_this.column_dimensions['BC'].width = coef * 6

ws_this.row_dimensions[1].height = 25
ws_this.row_dimensions[2].height = 34
for i in range(3, ws_this.max_row+1):
    ws_this.row_dimensions[i].height = 16.5

In [ ]:
from openpyxl.styles import Font, PatternFill, Alignment

In [ ]:
ws_this['A1'].value = str(datetime.now().month) + '/' + str(datetime.now().day)
ws_this['A1'].font = Font(b=True)
ws_this['A1'].number_format = 'yyyy/m/d;@'

ws_this['C1'].value = '每周二删除本周改价记录栏'
ws_this['C1'].font = Font(bold=True, color='FFFF0000')
ws_this['C1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['D1'].value = '基本信息'
ws_this['D1'].font = Font(b=True)
ws_this.merge_cells('D1:R1')

ws_this['S1'].value = '近7周 周销情况'
ws_this.merge_cells('S1:Y1')

ws_this['AA1'].value = '任务栏'
ws_this['AA1'].font = Font(b=True)
ws_this['AA1'].fill = PatternFill(patternType='solid', fgColor='FF92D050')

ws_this['AB1'].value = '=SUM(AB3:AB' + str(ws_this.max_row) + ')'
ws_this['AB1'].font = Font(b=True, color='FFFF0000')

ws_this['AC1'].value = '=SUM(AC3:AC' + str(ws_this.max_row) + ')'
ws_this['AC1'].font = Font(b=True, color='FFFF0000')

ws_this['AD1'].value = '=SUM(AD3:AD' + str(ws_this.max_row) + ')'
ws_this['AD1'].font = Font(b=True, color='FFFF0000')

ws_this['AE1'].value = '=SUM(AE3:AE' + str(ws_this.max_row) + ')'
ws_this['AE1'].font = Font(b=True, color='FFFF0000')

ws_this['AF1'].value = '=SUM(AF3:AF' + str(ws_this.max_row) + ')'
ws_this['AF1'].font = Font(b=True, color='FFFF0000')

ws_this['AG1'].value = '=SUM(AG3:AG' + str(ws_this.max_row) + ')'
ws_this['AG1'].font = Font(b=True, color='FFFF0000')

ws_this['AH1'].value = '=SUM(AH3:AH' + str(ws_this.max_row) + ')'
ws_this['AH1'].font = Font(b=True, color='FFFF0000')

ws_this['AR1'].value = '格式案例：10.19-改0; 9.23-T5; 9.30-H; 10.5-J3; 10.18-T6(日期，最新在后面）'

ws_this['AV1'].value = '实时库存≤5，周销不为0，或者周销为0，无record不为0的，在线都已改为实时库存，且已关闭自动补货'
ws_this['AV1'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

In [ ]:
for list_cell in ws_this['A2:R2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
for list_cell in ws_this['Z2:AA2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')
        
ws_this['AB2'].fill = PatternFill(patternType='solid', fgColor='FFFFFF00')

ws_this['AC2'].fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

ws_this['AD2'].fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AE2:AH2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FF00B0F0')

for list_cell in ws_this['AI2:AO2']:
    for cell in list_cell:
        cell.number_format = 'yyyy/m/d;@'
        cell.fill = PatternFill(patternType='solid', fgColor='FFFABF8E')

for list_cell in ws_this['AP2:AQ2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.fill = PatternFill(patternType='solid', fgColor='FF963634')

for list_cell in ws_this['AR2:AU2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

for list_cell in ws_this['AV2:AX2']:
    for cell in list_cell:
        cell.font = Font(color='FFFFFFFF')
        cell.fill = PatternFill(patternType='solid', fgColor='FFE26B0A')

for list_cell in ws_this['AY2:BA2']:
    for cell in list_cell:
        cell.fill = PatternFill(patternType='solid', fgColor='FF8DB4E2')

In [ ]:
for i in tqdm(range(3, ws_this.max_row+1), desc='Progress', ncols=77):
    ws_this['E' + str(i)].number_format = '\\$#,##0.00;\\-\\$#,##0.00'
    
    no = ws_this['BC' + str(i)].value
    if no == None:
        continue
    
    no = int(no) + 2
    if ws_last['AH' + str(no)].font.color != None and type(ws_last['AH' + str(no)].font.color.rgb) == str:
        ws_this['AH' + str(i)].font = Font(color=ws_last['AH' + str(no)].font.color.rgb)
    if ws_last['AG' + str(no)].font.color != None and type(ws_last['AG' + str(no)].font.color.rgb) == str:
        ws_this['AG' + str(i)].font = Font(color=ws_last['AG' + str(no)].font.color.rgb)
    if ws_last['AF' + str(no)].font.color != None and type(ws_last['AF' + str(no)].font.color.rgb) == str:
        ws_this['AF' + str(i)].font = Font(color=ws_last['AF' + str(no)].font.color.rgb)
    if ws_last['AE' + str(no)].font.color != None and type(ws_last['AE' + str(no)].font.color.rgb) == str:
        ws_this['AE' + str(i)].font = Font(color=ws_last['AE' + str(no)].font.color.rgb)
    if ws_last['AD' + str(no)].font.color != None and type(ws_last['AD' + str(no)].font.color.rgb) == str:
        ws_this['AD' + str(i)].font = Font(color=ws_last['AD' + str(no)].font.color.rgb)

In [ ]:
wb_this.save('./断货改价-2U-下午 ' + str(datetime.now().month) + '.' + str(datetime.now().day) + '.xlsx')
wb_this.close()
wb_last.close()

print('Done ~')